In [21]:
from logging import getLogger
import gokart
import logging
import luigi
import pickle
logger = getLogger(__name__)

In [30]:
class Sample(gokart.TaskOnKart):
    def run(self):
        self.dump('hello world')


class StringToSplit(gokart.TaskOnKart):
    task = gokart.TaskInstanceParameter()

    def run(self):
        sample = self.load('task')
        self.dump(sample.split(' '))


class Main(gokart.TaskOnKart):
    def requires(self):
        return StringToSplit(task=Sample())

Jupyter上で実行するときは`gokart.build`, コマンドで実行するときは`gokart.run`を使う
試しに`Sample`のみ実行してみる。
（コマンドで実行する場合は`python sample.py Sample --local-scheduler --param=hello`）  

※指定するのは最後のTaskで、そこから機械的に依存関係を見て実行してくれるイメージ

In [31]:
gokart.build(Sample(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Sample() is complete
INFO: Informed scheduler that task   Sample__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=652455159, workers=1, host=88071f242bda, username=root, pid=137) running   Sample()
INFO: [pid 137] Worker Worker(salt=652455159, workers=1, host=88071f242bda, username=root, pid=137) done      Sample()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   Sample__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=652455159, workers=1, host=88071f242bda, username=root, pid=137) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 Sample(...)

This progress looks :) because there

出力されるファイルを見てみる  
resources/  
├── __main__  
│   └── Example_8441c59b5ce0113396d53509f19371fb.pkl  
└── log  
    ├── module_versions  
    │   └── Example_8441c59b5ce0113396d53509f19371fb.txt  
    ├── processing_time  
    │   └── Example_8441c59b5ce0113396d53509f19371fb.pkl  
    ├── random_seed  
    │   └── Example_8441c59b5ce0113396d53509f19371fb.pkl  
    ├── task_log  
    │   └── Example_8441c59b5ce0113396d53509f19371fb.pkl  
    └── task_params  
        └── Example_8441c59b5ce0113396d53509f19371fb.pkl  

In [32]:
# タスクの出力結果
with open('resources/__main__/Sample_0364bf12abd58b1458027713a43be27e.pkl', 'rb') as f:
    print(pickle.load(f))

hello world


続いてMainタスクを実行してみる

In [33]:
gokart.build(Main(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Main() is complete
DEBUG: Checking if StringToSplit(task=Sample(cdf55a3d6c255d8c191f5f472da61f99)) is complete
INFO: Informed scheduler that task   Main__99914b932b   has status   PENDING
DEBUG: Checking if Sample() is complete
INFO: Informed scheduler that task   StringToSplit___type____Sample_0b7100a366   has status   PENDING
INFO: Informed scheduler that task   Sample__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=117981665, workers=1, host=88071f242bda, username=root, pid=137) running   StringToSplit(task=Sample(cdf55a3d6c255d8c191f5f472da61f99))
INFO: [pid 137] Worker Worker(salt=117981665, workers=1, host=88071f242bda, username=root, pid=137) done      StringToSplit(task=Sample(cdf55a3d6c255d8c191f5f472da61f99))
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   StringToSplit_

Sampleタスクは既に実行されているため、実行されていない。
StringToSplitの出力結果も見てみる。

In [36]:
with open('resources/__main__/StringToSplit_bdd05477115a331c4dce78999c46b7d2.pkl', 'rb') as f:
    print(pickle.load(f))

['hello', 'world']


上の方法だとSampleは再実行されなかったが、再実行させていときには2つの方法がある。
1. rerun parameterを使う
2. version parameterを使う

In [39]:
# rerun parameterを使う
class Sample(gokart.TaskOnKart):
    def run(self):
        self.dump('hello world')


class StringToSplit(gokart.TaskOnKart):
    task = gokart.TaskInstanceParameter()

    def run(self):
        sample = self.load('task')
        self.dump(sample.split(' '))


class Main(gokart.TaskOnKart):
    rerun=True
    
    def requires(self):
        return StringToSplit(task=Sample(rerun=self.rerun), rerun=self.rerun)

In [40]:
gokart.build(Sample(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Sample() is complete
INFO: Informed scheduler that task   Sample__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=254430108, workers=1, host=88071f242bda, username=root, pid=137) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 Sample(...)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



In [41]:
gokart.build(Main(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Main() is complete
DEBUG: Checking if StringToSplit(task=Sample(cdf55a3d6c255d8c191f5f472da61f99)) is complete
INFO: Informed scheduler that task   Main__99914b932b   has status   PENDING
DEBUG: Checking if Sample() is complete
INFO: Informed scheduler that task   StringToSplit___type____Sample_0b7100a366   has status   PENDING
INFO: Informed scheduler that task   Sample__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 137] Worker Worker(salt=436235229, workers=1, host=88071f242bda, username=root, pid=137) running   Sample()
INFO: [pid 137] Worker Worker(salt=436235229, workers=1, host=88071f242bda, username=root, pid=137) done      Sample()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   Sample__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] 

今度はSampleも再実行されている。

In [100]:
# 2.version parameterを使う
class Sample(gokart.TaskOnKart):
    __version = luigi.IntParameter(default=1)
    
    def run(self):
        self.dump('hello world')


class StringToSplit(gokart.TaskOnKart):
    task = gokart.TaskInstanceParameter()

    def run(self):
        sample = self.load('task')
        self.dump(sample.split(' '))


class Main(gokart.TaskOnKart):
    def requires(self):
        return StringToSplit(task=Sample(__version=2))

In [101]:
gokart.build(Sample(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Sample(_Sample__version=1) is complete
INFO: Informed scheduler that task   Sample_1_c68ef0739e   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=239108188, workers=1, host=88071f242bda, username=root, pid=137) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 Sample(...)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



In [44]:
gokart.build(Main(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if Main() is complete
DEBUG: Checking if StringToSplit(task=Sample(6d235ca0a7e4d46a49a87e0763459506)) is complete
INFO: Informed scheduler that task   Main__99914b932b   has status   PENDING
DEBUG: Checking if Sample(_Sample__version=1) is complete
INFO: Informed scheduler that task   StringToSplit___type____Sample_8e8329268e   has status   PENDING
INFO: Informed scheduler that task   Sample_1_c68ef0739e   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=616641627, workers=1, host=88071f242bda, username=root, pid=137) running   StringToSplit(task=Sample(6d235ca0a7e4d46a49a87e0763459506))
INFO: [pid 137] Worker Worker(salt=616641627, workers=1, host=88071f242bda, username=root, pid=137) done      StringToSplit(task=Sample(6d235ca0a7e4d46a49a87e0763459506))
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that ta

あれ、再実行されてない。よくわからんけどとりあえず先に進む。（__versionパラメタの値を変えれば別タスクになるってことっぽい）

***
## TaskOnKart

In [45]:
class TaskA(gokart.TaskOnKart):
    param = luigi.Parameter()

    def output(self):
        return self.make_target('output_of_task_a.pkl')

    def run(self):
        results = f'param={self.param}'
        self.dump(results)


class TaskB(gokart.TaskOnKart):
    param = luigi.Parameter()

    def requires(self):
        return TaskA(param='world')

    def output(self):
        return self.make_target('output_of_task_b.pkl')

    def run(self):
        output_of_task_a = self.load()
        results = f'Task A: {output_of_task_a}\nTaskB: param={self.param}'
        self.dump(results)

- `output`では各タスクの出力先と拡張子を指定できる（何も指定しない場合は`./resources/__main__/`にpklで出力される）。
- `make_model_target()`でさらに多様な拡張子を設定できるらしい。

In [47]:
gokart.build(TaskB(param='hello'), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if TaskB(param=hello) is complete
DEBUG: Checking if TaskA(param=world) is complete
INFO: Informed scheduler that task   TaskB_hello_523b4d23f7   has status   PENDING
INFO: Informed scheduler that task   TaskA_world_82a8b4cc97   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=552424809, workers=1, host=88071f242bda, username=root, pid=137) running   TaskA(param=world)
INFO: [pid 137] Worker Worker(salt=552424809, workers=1, host=88071f242bda, username=root, pid=137) done      TaskA(param=world)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   TaskA_world_82a8b4cc97   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=552424809, workers=1, host=88071f242bda, username=root, pid=137) running   TaskB(param=hello)
INFO: [pid 137] Worker Wo

出力先が変わっている

In [50]:
# TaskAとTaskBの出力
with open('resources/output_of_task_a_863d9ba1d171a05c64018f7c24792706.pkl', 'rb') as f:
    print(pickle.load(f))
with open('resources/output_of_task_b_cd5a550cf6d49634f4310bf55f5071c5.pkl', 'rb') as f:
    print(pickle.load(f))

param=world
Task A: param=world
TaskB: param=hello


### 複数のタスクから出力を受け取りたいとき

In [82]:
class TaskA(gokart.TaskOnKart):
    def run(self):
        self.dump('hello')


class TaskB(gokart.TaskOnKart):
    def run(self):
        self.dump('world')

class TaskC(gokart.TaskOnKart):
    def requires(self):
        return dict(a=TaskA(), b=TaskB())
    
    def run(self):
        input_from_a = self.load('a')
        input_from_b = self.load('b')
        self.dump(f'{input_from_a}, {input_from_b}')

In [83]:
gokart.build(TaskC(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if TaskC() is complete
INFO: Informed scheduler that task   TaskC__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=686370457, workers=1, host=88071f242bda, username=root, pid=137) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 TaskC(...)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



In [53]:
# TaskCの出力
with open('resources/__main__/TaskC_7f9110ac41c35edd2c9aba2eead98977.pkl', 'rb') as f:
    print(pickle.load(f))

hello, world


### 複数の出力を渡したいとき

In [75]:
class TaskA(gokart.TaskOnKart):
    def output(self):
        return dict(a=self.make_target('output_a.pkl'), b=self.make_target('output_b.pkl'))
    
    def run(self):
        self.dump('huuuu', 'a')
        self.dump('hoooo', 'b')


class TaskB(gokart.TaskOnKart):
    def requires(self):
        return TaskA(rerun=True)
    
    def run(self):
        imput1 = self.load('a')
        imput2 = self.load('b')
        self.dump(f'{imput1}, {imput2}')

In [76]:
gokart.build(TaskB(rerun=True), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if TaskB() is complete
DEBUG: Checking if TaskA() is complete
INFO: Informed scheduler that task   TaskB__99914b932b   has status   PENDING
INFO: Informed scheduler that task   TaskA__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=731295791, workers=1, host=88071f242bda, username=root, pid=137) running   TaskA()
INFO: [pid 137] Worker Worker(salt=731295791, workers=1, host=88071f242bda, username=root, pid=137) done      TaskA()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   TaskA__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=731295791, workers=1, host=88071f242bda, username=root, pid=137) running   TaskB()
INFO: [pid 137] Worker Worker(salt=731295791, workers=1, host=88071f242bda, username=root, pid=

In [77]:
with open('resources/__main__/TaskB_8258602061b8d7093066089f6274ce34.pkl', 'rb') as f:
    print(pickle.load(f))

huuuu, hoooo


### pandasのdfを扱うのに便利な機能

`SamplePandasTypeConfig`を書いておくと、カラムに違うデータ型が入りそうなときエラーになってくれる

In [84]:
import pandas as pd
class SamplePandasTypeConfig(gokart.PandasTypeConfig):

    @classmethod
    def type_dict(cls):
        return {'int_column': int}


class SampleTask(gokart.TaskOnKart):

    def run(self):
        df = pd.DataFrame(dict(int_column=['a']))
        self.dump(df)

In [85]:
gokart.build(SampleTask(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if SampleTask() is complete
INFO: Informed scheduler that task   SampleTask__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=640015612, workers=1, host=88071f242bda, username=root, pid=137) running   SampleTask()
INFO: [pid 137] Worker Worker(salt=640015612, workers=1, host=88071f242bda, username=root, pid=137) done      SampleTask()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   SampleTask__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=640015612, workers=1, host=88071f242bda, username=root, pid=137) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 SampleTask(...)

This progres

In [86]:
with open('resources/__main__/SampleTask_5d77233c7f4ad76106e4f3c66b5d29d7.pkl', 'rb') as f:
    print(pickle.load(f))

  int_column
0          a


って思ったけどエラーにならないじゃん。なぜ？？

dfを読み込むときには`load_data_frame()`が使える。カラムを指定すれば、reset_indexとdropを読み込み次に自動でやってくれる。

In [87]:
class CreateDf(gokart.TaskOnKart):
    def run(self):
        df = pd.DataFrame(dict(a=['a'], b=['b'], c=['c']))
        self.dump(df)

class LoadDf(gokart.TaskOnKart):
    def requires(self):
        return CreateDf()

    def run(self):
        df = self.load_data_frame(required_columns={'a', 'c'}, drop_columns=True)
        self.dump(df)

In [88]:
gokart.build(LoadDf(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if LoadDf() is complete
DEBUG: Checking if CreateDf() is complete
INFO: Informed scheduler that task   LoadDf__99914b932b   has status   PENDING
INFO: Informed scheduler that task   CreateDf__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=320155889, workers=1, host=88071f242bda, username=root, pid=137) running   CreateDf()
INFO: [pid 137] Worker Worker(salt=320155889, workers=1, host=88071f242bda, username=root, pid=137) done      CreateDf()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   CreateDf__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=320155889, workers=1, host=88071f242bda, username=root, pid=137) running   LoadDf()
INFO: [pid 137] Worker Worker(salt=320155889, workers=1, host=88071f242bda, u

In [89]:
with open('resources/__main__/LoadDf_73a10e28a2a45c6791641134106e3ef4.pkl', 'rb') as f:
    print(pickle.load(f))

   a  c
0  a  c


`dump`でなく`fail_on_empty_dump`を使えば、空のdfの時エラーになる

In [95]:
class CreateFmptyDf(gokart.TaskOnKart):
    def run(self):
        df = pd.DataFrame()
        self.fail_on_empty_dump(df)

In [97]:
gokart.build(CreateFmptyDf(rerun=True), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if CreateFmptyDf() is complete
INFO: Informed scheduler that task   CreateFmptyDf__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=487947014, workers=1, host=88071f242bda, username=root, pid=137) running   CreateFmptyDf()
ERROR: [pid 137] Worker Worker(salt=487947014, workers=1, host=88071f242bda, username=root, pid=137) failed    CreateFmptyDf()
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/luigi/worker.py", line 191, in run
    new_deps = self._run_get_new_deps()
  File "/opt/anaconda3/lib/python3.7/site-packages/luigi/worker.py", line 133, in _run_get_new_deps
    task_gen = self.task.run()
  File "<ipython-input-95-228130c97141>", line 4, in run
    self.fail_on_empty_dump(df)
TypeError: 'bool' object is not callable
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Info

GokartBuildError: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 failed:
    - 1 CreateFmptyDf(...)

This progress looks :( because there were failed tasks

===== Luigi Execution Summary =====


## Task Parameters

- Luigi ParameterとGokart Parameterがある
    - Luigiの方は普通のデータ型を渡したいときに使う
    - Gokartの方はGokartのインスタンスを渡すときに使う

In [98]:
class TaskA(gokart.TaskOnKart):
    def run(self):
        self.dump('Hello')


class TaskB(gokart.TaskOnKart):
    require_task = gokart.TaskInstanceParameter()

    def requires(self):
        return self.require_task

    def run(self):
        task_a = self.load()
        self.dump(','.join([task_a, 'world']))

In [99]:
gokart.build(TaskB(require_task=TaskA()), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if TaskB(require_task=TaskA(c23680bed2ec8ff44c0c58b49bd72f35)) is complete
DEBUG: Checking if TaskA() is complete
INFO: Informed scheduler that task   TaskB___type____TaskA__0574fcdfe8   has status   PENDING
INFO: Informed scheduler that task   TaskA__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=512840982, workers=1, host=88071f242bda, username=root, pid=137) running   TaskB(require_task=TaskA(c23680bed2ec8ff44c0c58b49bd72f35))
INFO: [pid 137] Worker Worker(salt=512840982, workers=1, host=88071f242bda, username=root, pid=137) done      TaskB(require_task=TaskA(c23680bed2ec8ff44c0c58b49bd72f35))
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   TaskB___type____TaskA__0574fcdfe8   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at

### パラメタの指定方法

- command lineから実行するときにオプションで指定する
- configファイルに書く（この場合環境変数使うこともできる）
- 上流タスクで指定する
- Inherit parameterで継承する

In [105]:
from gokart import inherits_config_params

# Inherit parameterで継承する
class MasterConfig(luigi.Config):
    param: str = luigi.Parameter()
    param2: str = luigi.Parameter()

@inherits_config_params(MasterConfig)
class SomeTask(gokart.TaskOnKart):
    param: str = luigi.Parameter()

ImportError: cannot import name 'inherits_config_params' from 'gokart' (/opt/anaconda3/lib/python3.7/site-packages/gokart/__init__.py)

### その他

ランダムシードを固定できる

In [108]:
"""
gokart.build(
    Task(
        fix_random_seed_methods=[
            "random.seed",
            "numpy.random.seed",
            "torch.random.manual_seed"],
        fix_random_seed_value=57))
"""

'\ngokart.build(\n    Task(\n        fix_random_seed_methods=[\n            "random.seed",\n            "numpy.random.seed",\n            "torch.random.manual_seed"],\n        fix_random_seed_value=57))\n'

### ちょっと複雑な例を試す

In [116]:
class TaskD(gokart.TaskOnKart):
    def output(self):
        return dict(h=self.make_target('output_h.pkl'), w=self.make_target('output_w.pkl'))
    
    def run(self):
        self.dump('hello', 'h')
        self.dump('world', 'w')


class TaskE(gokart.TaskOnKart):
    def run(self):
        self.dump('!')

class TaskF(gokart.TaskOnKart):
    def requires(self):
        return dict(a=TaskD(), b=TaskE())
    
    def run(self):
        input = self.load('a')
        self.dump(f'{input}')

In [117]:
gokart.build(TaskF(), return_value=False, log_level=logging.DEBUG)

DEBUG: Checking if TaskF() is complete
DEBUG: Checking if TaskD() is complete
DEBUG: Checking if TaskE() is complete
INFO: Informed scheduler that task   TaskF__99914b932b   has status   PENDING
INFO: Informed scheduler that task   TaskE__99914b932b   has status   DONE
INFO: Informed scheduler that task   TaskD__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 137] Worker Worker(salt=896943363, workers=1, host=88071f242bda, username=root, pid=137) running   TaskD()
INFO: [pid 137] Worker Worker(salt=896943363, workers=1, host=88071f242bda, username=root, pid=137) done      TaskD()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   TaskD__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 137] Worker Worker(salt=896943363, workers=1, host=88071f242bda, username=root, pid=137)

In [118]:
with open('resources/__main__/TaskF_4cee666e2efd15bcd0febcdea089bdf3.pkl', 'rb') as f:
    print(pickle.load(f))

{'h': 'hello', 'w': 'world'}
